In [61]:
import os
import sys
import pandas as pd

In [62]:
from global_functions import up_load_shp, make_point, add_geo_info_shp,add_geo_info_gdb

In [63]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [64]:
TAZ_V4_date='240404'

In [65]:
software_folder_location=r'W:\Data\Forecast\Tools\forecast_git\create_forecast_basic'

In [66]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_{}.shp'.format(software_folder_location, TAZ_V4_date))

In [67]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'jeru_metro_jtmt_border_240402', 'jeru_metro')

In [68]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'in_jeru', 'in_jerusal')
taz.loc[taz['in_jerusal']==0,'in_jerusal']='no'

In [69]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'main_secto', 'main_secto')

In [70]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'SCHN_NAME', 'SCHN_NAME')

In [71]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'subdistrict2008','ENG_NAME_n')
taz['zonetype']=taz['ENG_NAME_n']

In [72]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'muni_under_JTMT_ITM', 'Muni_Heb')

In [73]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'muni_under_JTMT_ITM', 'Sug_Muni')

In [74]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'muni_under_JTMT_ITM', 'CR_PNIM')

In [75]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'highBusinessFlag', 'HighBiz')
taz.rename(columns={'HighBiz': 'highBusinessFlag'}, inplace=True)

In [76]:
taz=add_geo_info_shp(taz,taz,software_folder_location,'schDistrict','ID').rename(columns={'ID':'SCHOOLDISTRICT'})

In [77]:
taz=add_geo_info_shp(taz,taz,software_folder_location,'urban','Urban')

In [78]:
taz=add_geo_info_shp(taz,taz,software_folder_location,'EIProp','EIProp')

In [79]:
col=['poly_puma', 'F3', 'F2', 'F1']

for c in col:
    taz=add_geo_info_shp(taz,taz,software_folder_location,'poly_pumas',c)
    

In [80]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

for c in col:
    taz=add_geo_info_shp(taz,taz,software_folder_location,'emp_category_type',c)
    taz['{}'.format(c)]=taz['{}'.format(c)]/100
    

In [81]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'commuting_230712', 'commuting')
taz['commuting']=taz['commuting']/100

In [82]:
taz['ieold']=0
taz.loc[taz['jeru_metro']==1,'ieold']=taz['commuting']
taz['IEProp']=taz['ieold']

In [83]:
taz['perScaled']=1-taz['EIProp']

In [84]:
col_parking=['FreeBuffer',
'PaidBuffer',
'Rest_EmpBuffer',
'searchtime',
'walktime',
'cost']

for c in col_parking:
    taz=add_geo_info_gdb(taz,taz,software_folder_location,'parking_abm',c,c)

In [85]:
taz.loc[taz['Taz_num'] < 7001, 'Agg_taz_nu'] = taz['Taz_num'] // 100
taz.loc[taz['Taz_num'] >= 7001, 'Agg_taz_nu'] = taz['Taz_num'] // 10

In [86]:
col_with_fix_num=['area',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCons',
 'county',
 'majunivenr',
 'parktot',
 'superZone',
 'Taz1','slope','REGION']


In [87]:

taz[col_with_fix_num]=1

In [88]:
taz['yosh']=0

taz.loc[taz['zonetype']=='Judea and Samaria','yosh']=1

taz['jerusalem_city']=0

taz.loc[taz['in_jerusal']=='yes','jerusalem_city']=1

In [89]:
taz.to_excel(r"{}\background_files\TAZ_V4_{}_with_geo_info.xlsx".format(software_folder_location, file_date), index=False)